# Creating a data set for YOLOV11 - TACO data set

- I want to work on 90 images, each containing annotations for: plastic straw and glass bottle

In [169]:
import json

# Open and read the JSON file
with open('datasets\\taco_dataset\\annotations.json', 'r') as file:
    data = json.load(file)

In [170]:
data.keys()

dict_keys(['info', 'images', 'annotations', 'scene_annotations', 'licenses', 'categories', 'scene_categories'])

In [171]:
data['categories']

[{'supercategory': 'Aluminium foil', 'id': 0, 'name': 'Aluminium foil'},
 {'supercategory': 'Battery', 'id': 1, 'name': 'Battery'},
 {'supercategory': 'Blister pack', 'id': 2, 'name': 'Aluminium blister pack'},
 {'supercategory': 'Blister pack', 'id': 3, 'name': 'Carded blister pack'},
 {'supercategory': 'Bottle', 'id': 4, 'name': 'Other plastic bottle'},
 {'supercategory': 'Bottle', 'id': 5, 'name': 'Clear plastic bottle'},
 {'supercategory': 'Bottle', 'id': 6, 'name': 'Glass bottle'},
 {'supercategory': 'Bottle cap', 'id': 7, 'name': 'Plastic bottle cap'},
 {'supercategory': 'Bottle cap', 'id': 8, 'name': 'Metal bottle cap'},
 {'supercategory': 'Broken glass', 'id': 9, 'name': 'Broken glass'},
 {'supercategory': 'Can', 'id': 10, 'name': 'Food Can'},
 {'supercategory': 'Can', 'id': 11, 'name': 'Aerosol'},
 {'supercategory': 'Can', 'id': 12, 'name': 'Drink can'},
 {'supercategory': 'Carton', 'id': 13, 'name': 'Toilet tube'},
 {'supercategory': 'Carton', 'id': 14, 'name': 'Other carton'

In [172]:
# categories
data['annotations'][0]

# id : 12 -> drink can 
# id: 6 -> glass bottle

{'id': 1,
 'image_id': 0,
 'category_id': 6,
 'segmentation': [[561.0,
   1238.0,
   568.0,
   1201.0,
   567.0,
   1175.0,
   549.0,
   1127.0,
   538.0,
   1089.0,
   519.0,
   1043.0,
   517.0,
   1005.0,
   523.0,
   964.0,
   529.0,
   945.0,
   520.0,
   896.0,
   525.0,
   862.0,
   536.0,
   821.0,
   554.0,
   769.0,
   577.0,
   727.0,
   595.0,
   678.0,
   596.0,
   585.0,
   588.0,
   346.0,
   581.0,
   328.0,
   569.0,
   306.0,
   570.0,
   276.0,
   576.0,
   224.0,
   560.0,
   205.0,
   564.0,
   170.0,
   578.0,
   154.0,
   608.0,
   136.0,
   649.0,
   127.0,
   688.0,
   127.0,
   726.0,
   129.0,
   759.0,
   141.0,
   784.0,
   153.0,
   792.0,
   177.0,
   788.0,
   193.0,
   782.0,
   209.0,
   792.0,
   238.0,
   802.0,
   271.0,
   802.0,
   294.0,
   791.0,
   319.0,
   789.0,
   360.0,
   794.0,
   395.0,
   810.0,
   529.0,
   819.0,
   609.0,
   841.0,
   675.0,
   882.0,
   728.0,
   916.0,
   781.0,
   928.0,
   802.0,
   938.0,
   834.0,
   940.0,
  

In [173]:

# Category IDs we're interested in

target_categories = {
    12: {'ann': [], 'img_ids': set()}, # drink can 
    6: {'ann': [], 'img_ids': set()}   # glass bottle
}

for annotation in data['annotations']:
    img_id = annotation['image_id']
    category_id = annotation['category_id']

    if category_id in target_categories:
        target_categories[category_id]['ann'].append(annotation)
        target_categories[category_id]['img_ids'].add(img_id)

# Example access:
print("Count of images with drink cans:")
print(len(target_categories[12]['img_ids']))
print("Count of images with glass bottles:")
print(len(target_categories[6]['img_ids']))

Count of images with drink cans:
151
Count of images with glass bottles:
74


In [174]:
# removing images that have annotaitons from both categories
common_img_ids = target_categories[6]['img_ids'] & target_categories[12]['img_ids']
print(common_img_ids)
target_categories[12]['img_ids'].difference_update(common_img_ids)
target_categories[6]['img_ids'].difference_update(common_img_ids)
print(common_img_ids)

{764, 582, 1320, 460, 116, 860}
{764, 582, 1320, 460, 116, 860}


In [175]:
target_categories[12]['img_ids'] = list(target_categories[12]['img_ids'])[:50]
target_categories[6]['img_ids'] = list(target_categories[6]['img_ids'])[:50]



In [176]:
# Create a new list with valid annotations for category 12
valid_annotations_cat_12 = [annotation for annotation in target_categories[12]['ann']
                            if annotation['image_id'] in target_categories[12]['img_ids']]

# Update the annotations list
target_categories[12]['ann'] = valid_annotations_cat_12

# Create a new list with valid annotations for category 6
valid_annotations_cat_6 = [annotation for annotation in target_categories[6]['ann']
                           if annotation['image_id'] in target_categories[6]['img_ids']]

# Update the annotations list
target_categories[6]['ann'] = valid_annotations_cat_6

# Check the lengths of the valid annotations
len(target_categories[12]['ann']), len(target_categories[6]['ann'])


(83, 71)

In [177]:
# creating final rows of data
images = data['images']
final_images = []
# added this additional parameter to make it easier for splitting, as there are no images that have annotations of differnt types
for row in images:
    if row['id'] in target_categories[12]['img_ids']:
        row['category_id'] = 1 

    if row['id'] in target_categories[6]['img_ids']:
        row['category_id'] = 2

    if row['id'] in target_categories[12]['img_ids'] or row['id'] in target_categories[6]['img_ids']:
        final_images.append(row)

final_annotations = target_categories[12]['ann'] + target_categories[6]['ann'] 

final_annotations = sorted(final_annotations, key=lambda x: x['id'])
final_images = sorted(final_images, key=lambda x: x['id'])

 

In [178]:
# mapping annotations to iamges

hierarchy = {}

i = 1
j = 1

for image in final_images:
    id_image = image['id']
    hierarchy[id_image] = [] 

    for annotation in final_annotations:
        id_ann = annotation['image_id'] 
        
        if id_image == id_ann:

            annotation['image_id'] = i
            annotation['id'] = j
            if annotation['category_id'] == 12:
                annotation['category_id'] = 1

            if annotation['category_id'] == 6:
                annotation['category_id'] = 2

            hierarchy[id_image].append(annotation)
            
            j += 1 

    hierarchy[i] = hierarchy[id_image]
    del hierarchy[id_image]

    i += 1

In [179]:
hierarchy.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100])

In [180]:
for idx, image in enumerate(final_images, start=1):
    image['file_name'] = f'img{idx}.jpg'
    image['id'] = idx

## Creating the custom COCO data set

In [181]:
final_data = {
    'info' : {'year': 2025,
            'version': None,
            'description': 'Project_Object_Detection',
            'contributor': None,
            'url': None,
            'date_created': '2025-03-16'},
    'images' : [],
    'annotations': [],
    'licenses' : [],
    'categories' : [{'supercategory': 'Can', 'id': 1, 'name': 'Drink can'},
                    {'supercategory': 'Bottle', 'id': 2, 'name': 'Glass bottle'}],
}

In [182]:
final_data['images'] = final_images
for key, value in hierarchy.items():
    for ann in hierarchy[key]:
        del ann['segmentation']
        del ann['iscrowd']
        del ann['area']
        final_data['annotations'].append(ann)

In [183]:
print(final_data['images'])

[{'id': 1, 'width': 1537, 'height': 2049, 'file_name': 'img1.jpg', 'license': None, 'flickr_url': 'https://farm66.staticflickr.com/65535/33978196618_e30a59e0a8_o.png', 'coco_url': None, 'date_captured': None, 'flickr_640_url': 'https://farm66.staticflickr.com/65535/33978196618_632623b4fc_z.jpg', 'category_id': 2}, {'id': 2, 'width': 2049, 'height': 1537, 'file_name': 'img2.jpg', 'license': None, 'flickr_url': 'https://farm66.staticflickr.com/65535/47803331492_0e1085ca55_o.png', 'coco_url': None, 'date_captured': None, 'flickr_640_url': 'https://farm66.staticflickr.com/65535/47803331492_b8c0e5aafe_z.jpg', 'category_id': 1}, {'id': 3, 'width': 1537, 'height': 2049, 'file_name': 'img3.jpg', 'license': None, 'flickr_url': 'https://farm66.staticflickr.com/65535/33978199868_88ee160849_o.png', 'coco_url': None, 'date_captured': None, 'flickr_640_url': 'https://farm66.staticflickr.com/65535/33978199868_1bc379170a_z.jpg', 'category_id': 1}, {'id': 4, 'width': 1537, 'height': 2049, 'file_name': 

In [184]:
print(final_data['annotations'])

[{'id': 1, 'image_id': 1, 'category_id': 2, 'bbox': [517.0, 127.0, 447.0, 1322.0]}, {'id': 2, 'image_id': 2, 'category_id': 1, 'bbox': [589.0, 548.0, 341.0, 405.0]}, {'id': 3, 'image_id': 3, 'category_id': 1, 'bbox': [556.0, 944.0, 490.0, 336.0]}, {'id': 4, 'image_id': 4, 'category_id': 2, 'bbox': [379.0, 1127.0, 425.0, 318.0]}, {'id': 5, 'image_id': 5, 'category_id': 1, 'bbox': [672.0, 1069.0, 219.0, 161.0]}, {'id': 6, 'image_id': 6, 'category_id': 2, 'bbox': [211.0, 1.0, 417.0, 1417.0]}, {'id': 7, 'image_id': 7, 'category_id': 1, 'bbox': [522.0, 1168.0, 240.0, 122.0]}, {'id': 8, 'image_id': 8, 'category_id': 1, 'bbox': [503.0, 808.0, 445.0, 357.0]}, {'id': 9, 'image_id': 9, 'category_id': 1, 'bbox': [399.0, 502.0, 483.0, 741.0]}, {'id': 10, 'image_id': 10, 'category_id': 2, 'bbox': [852.0, 638.0, 687.0, 291.0]}, {'id': 11, 'image_id': 11, 'category_id': 2, 'bbox': [647.0, 1033.0, 820.0, 376.0]}, {'id': 12, 'image_id': 12, 'category_id': 1, 'bbox': [1015.0, 979.0, 190.0, 214.0]}, {'id

In [185]:
# Save to JSON file
with open('datasets\\coco_dataset\\annotations_coco.json', 'w') as f:
    json.dump(final_data, f)